# Bayesian Approach

Introduction: 

 After examining the result of GLM model, we found season, time period (peak hours),weather and traffic day have some influence on the number of deaths. Plus, those variables also have interaction with other variables, such as speed, obstruction etc.

As such, we decided to use hierarchical model to select weather, season, etc as main effect, and make speed, sequence as subject level effect. Because we might see our data in this way, speed of a vehicle is an individual variable, but it has some dependence on the weather, season.

This assumption is qualified for the application of hierarchical model. The units of analysis are usually individuals (at a lower level) who are nested within contextual/aggregate units (at a higher level).

## 1.1 Load several relevant libraries 

In [1]:
library(dplyr)
library(readr)
library(magrittr)
library(ggplot2)
library(brms)   # loading various libraries


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: Rcpp
Loading 'brms' package (version 1.9.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').
Plotting theme set to bayesplot::theme_default().


## 1.2 Load data and drop irrelevant columns

In [2]:
brms_data <- read.csv("2015 US TRAFFIC FATALS BRMS APR24.csv")

In [3]:
head(brms_data,3)

FATALS,STATE,ST_CASE,Sequence,ROLLOVER,SPEED,Obstruction,DRUNK_DR,VE_TOTAL,VE_FORMS,...,noon,afternoon,LATITUDE,LONGITUD,Clear,Rain,Hail,Snow,Fog,Blowing_sand
2,1,10025,4,0,95,0,1,1,1,...,0,0,33.08891,-87.67460,1,0,0,0,0,0
1,1,10029,3,0,65,0,0,1,1,...,0,0,31.16131,-85.19056,0,1,0,0,0,0
5,1,10030,5,1,96,0,0,2,2,...,1,0,32.78083,-86.47020,1,0,0,0,0,0


In [4]:
drop.cols = c("ST_CASE", "STATE")
brms_data <- brms_data %>% select(-one_of(drop.cols))
# c...string %>% -?  #drop columns

In [5]:
head(brms_data,3)

FATALS,Sequence,ROLLOVER,SPEED,Obstruction,DRUNK_DR,VE_TOTAL,VE_FORMS,PERMVIT,Principal.Arterial,...,noon,afternoon,LATITUDE,LONGITUD,Clear,Rain,Hail,Snow,Fog,Blowing_sand
2,4,0,95,0,1,1,1,2,0,...,0,0,33.08891,-87.67460,1,0,0,0,0,0
1,3,0,65,0,0,1,1,1,1,...,0,0,31.16131,-85.19056,0,1,0,0,0,0
5,5,1,96,0,0,2,2,5,0,...,1,0,32.78083,-86.47020,1,0,0,0,0,0


## 2.1 Summary grouping parameters- Weather

In [6]:
levels(brms_data$Weather)
summary(brms_data$Weather)

[1] "Blowing_sand" "Clear"        "Fog"          "Hail"         "Other"       
[6] "Rain"         "Snow"

Blowing_sand        Clear          Fog         Hail        Other         Rain 
          11         7451          112           31         1372          650 
        Snow 
          96

"""mydf$med <- `levels<-`(factor(mydf$med), list("foo"=0, "bar"=c(1,2)))
mydf
is.factor(mydf$med)

In [7]:
table(brms_data$Weather, brms_data$Season) 
#cross table to check which subgroup should be collapsed

              
               Fall Spring Summer Winter
  Blowing_sand    1      6      2      2
  Clear        2110   1653   2080   1608
  Fog            21     22     14     55
  Hail            1     11      1     18
  Other         328    329    345    370
  Rain          161    132    138    219
  Snow            0     53      1     42

## 2.2 Collapse some categories within Weather

In [8]:
brms_data$Weather <- `levels<-`(factor(brms_data$Weather), list("Other"=c("Other","Blowing_sand","Hail"),
                                                                "Fog"=c("Fog"),"Snow"=c("Snow"),
                                                                "Rain"=c("Rain"),"Clear"=c("Clear")))
#we recode "Other" to incorporate more categories and keep the remaining categories

## 2.3 Change the columns into factor variables

In [9]:
brms_data$Season <- factor(brms_data$Season)
brms_data$Region <- factor(brms_data$Region)
brms_data$Day <- factor(brms_data$Day)
brms_data$period <- factor(brms_data$period)
#$ dollar sign here mens extracting some columns from the table
#convert to factor var for being grouping parameter

In [10]:
is.factor(brms_data$Weather)
is.factor(brms_data$Day)   #check the nature of columns, whether it is factor variable
is.factor(brms_data$period)

[1] TRUE

[1] TRUE

[1] TRUE

In [11]:
levels(brms_data$Weather)
summary(brms_data$Weather) # already inclueded

[1] "Other" "Fog"   "Snow"  "Rain"  "Clear"

Other   Fog  Snow  Rain Clear 
 1414   112    96   650  7451

## 3.1 Create crosstap table to summary interacted parameters

In [12]:
table(brms_data$Weather, brms_data$Season) # we are trying to create an intersected column with season and weather

       
        Fall Spring Summer Winter
  Other  330    346    348    390
  Fog     21     22     14     55
  Snow     0     53      1     42
  Rain   161    132    138    219
  Clear 2110   1653   2080   1608

In [13]:
table(brms_data$Day, brms_data$period)

             
              nonpeak peak
  Normal Day     4026 2035
  Traffic Day    2218 1444

## 3.2 Create interaction variable

In [14]:
brms_data$Weather_season <-interaction(brms_data$Weather, brms_data$Season)
# create weather_season factor var to exclude snow.fall and snow.summer subgroup

In [15]:
names(brms_data)
brms_data$Weather_season <- factor(brms_data$Weather_season)
is.factor(brms_data$Weather_season)
nrow(brms_data)

[1] "FATALS"             "Sequence"           "ROLLOVER"          
 [4] "SPEED"              "Obstruction"        "DRUNK_DR"          
 [7] "VE_TOTAL"           "VE_FORMS"           "PERMVIT"           
[10] "Principal.Arterial" "Minor.Arterial"     "Collector"         
[13] "Intersection"       "Daylight"           "Region"            
[16] "Season"             "Weather"            "Day"               
[19] "period"             "Winter"             "Spring"            
[22] "Summer"             "WEEKDAY"            "SAT"               
[25] "morning_peak"       "night_peak"         "after_work"        
[28] "noon"               "afternoon"          "LATITUDE"          
[31] "LONGITUD"           "Clear"              "Rain"              
[34] "Hail"               "Snow"               "Fog"               
[37] "Blowing_sand"       "Weather_season"

[1] TRUE

[1] 9723


summary(brms_data$Weather)
counts <- table(brms_data$Weather)
barplot(counts, main="Weather Distribution", 
  	xlab="Various Weather Situations")


In [16]:
summary(brms_data$Weather_season) 
# summary before delete some subgroup of weather_season
# the next step would be remove both record and category(factor var) from weather_season

Other.Fall     Fog.Fall    Rain.Fall   Clear.Fall Other.Spring   Fog.Spring 
         330           21          161         2110          346           22 
 Snow.Spring  Rain.Spring Clear.Spring Other.Summer   Fog.Summer  Snow.Summer 
          53          132         1653          348           14            1 
 Rain.Summer Clear.Summer Other.Winter   Fog.Winter  Snow.Winter  Rain.Winter 
         138         2080          390           55           42          219 
Clear.Winter 
        1608

## 3.3 Drop the subgroups that have very few data (Snow.summer & Snow.fall)  

#brms_matrix<-as.matrix(brms_data)

"""head(brms_matrix,5)
is.matrix(brms_matrix)
is.matrix(brms_data)

In [17]:
#result<-filter(brms_data,brms_data$Weather_season=="Snow.Summer")
#nrow(result)
#brms_data<-brms_data[!(brms_data$Weather_season=="Snow.Summer"|brms_data$Weather_season=="Snow.Fall"),]

# delete rows (the record), but still have the category under weather_season


nrow(brms_data[brms_data$Weather_season != "Snow.Summer", , drop=FALSE])

brms_data<-brms_data[brms_data$Weather_season != "Snow.Summer", , drop=FALSE]

brms_data$Weather_season<-factor(brms_data$Weather_season)

nrow(brms_data)

summary(brms_data$Weather_season) 

# here, the recoding has completed


[1] 9722

[1] 9722

Other.Fall     Fog.Fall    Rain.Fall   Clear.Fall Other.Spring   Fog.Spring 
         330           21          161         2110          346           22 
 Snow.Spring  Rain.Spring Clear.Spring Other.Summer   Fog.Summer  Rain.Summer 
          53          132         1653          348           14          138 
Clear.Summer Other.Winter   Fog.Winter  Snow.Winter  Rain.Winter Clear.Winter 
        2080          390           55           42          219         1608

In [18]:
#brms_data$Weather_season <- `levels<-`(factor(brms_data$Weather_season), list("Snow.Spring"=c("Snow.Spring","Snow.Summer")))

                                                             

## 4 Split the dataset into training and validation parts

In [19]:
brsmp_size <- floor(0.7*nrow(brms_data))
set.seed(12345)
brtrain_ind <- sample(seq_len(nrow(brms_data)), size = brsmp_size)
brtraining.data <- brms_data[brtrain_ind,]
brtesting.data <- brms_data[-brtrain_ind,]   # splitting the dataset into training and validation subset

In [20]:
head(brtraining.data,3)

,FATALS,Sequence,ROLLOVER,SPEED,Obstruction,DRUNK_DR,VE_TOTAL,VE_FORMS,PERMVIT,Principal.Arterial,...,afternoon,LATITUDE,LONGITUD,Clear,Rain,Hail,Snow,Fog,Blowing_sand,Weather_season
7010,1,3,0,55,0,0,2,2,2,0,...,0,38.87434,-83.88585,1,0,0,0,0,0,Clear.Winter
8515,1,4,1,44,0,1,1,1,1,0,...,1,43.29648,-103.38934,1,0,0,0,0,0,Clear.Fall
7398,1,1,0,35,0,0,1,1,1,1,...,1,40.43500,-79.99864,1,0,0,0,0,0,Clear.Fall


## 5 Run Bayesian model 

Explanation for the model formula:

1)
Sequence is under the interaction between traffic day and traffic hour
SPEED is under the Weather_season level
Obstruction is under the weather effect
Interaction between Drunk driver and Person on the moving vehicles

2)
Sequence, SPEED and Obstruction are also included in the population level 

In [ ]:

bayes_model <- brm(FATALS ~ (Sequence|Day:period) + (1+SPEED | Weather_season) + (Obstruction|Weather) + 
                   DRUNK_DR : PERMVIT+ Sequence + SPEED +Obstruction, data=brtraining.data,family=poisson(link = "log"),
                  chains=6,iter=5500)
summary(bayes_model)

# : means interaction, |followed by the grouping parameters, before running the hierarchy model, we should
# first define the factor variables for the grouping parameters
#the result should have the rhat coverge to 1, whereas right now it is lack of chains and iterations

#by holding other independent variables constant, one unit change in var (SPEED) will change the strd of posterior distribution 
#of the fatalities number by *** units---estimate---rhat converge to 1
# Rhat might never converge to one, cuz the subset has very lower data content


#putting Sequence.etc out of the bracket means population level

Compiling the C++ model
Start sampling
